## Definning [Uborn,Living,Died] Non-Deterministic Finite automaton for 'n' events/Intervals.

### Basic Assumptions:
1. 0: Unborn
2. 1: Living
3. 2: Died

In [1]:
'''
Initializing all pre-requisite packages.
'''
from tkinter import *
from tkinter import font as tkFont
from graphviz import Digraph
import  tkinter as tk
from PIL import ImageTk, Image
from pdf2image import convert_from_path
from graphviz import Source
from automata.fa.nfa import NFA
from visual_automata.fa.nfa import VisualNFA
import os

'''
Function definitions to construct a {U, L, D} Automaton.
'''

# Definning Final and Initial State based on Number of Intervals.
def final_state(n):
    temp = [] #Temporary Storage list to hold final state of an automaton.
    for i in range(n):
        temp += [2] #Appending '2' for each intervals signifying the event is died/finished.
    return temp

# Definning Previous State based on Current State. [Making use of Recursion function]
def prev_state(n,q):  # n=number of intervals, q=string of length n over [0,1,2]
    temp = []
    if q[0] == 1:
        temp.append([0]+q[1:])
    elif q[0] == 2:
        temp.append([1]+q[1:])
    if n > 1:
        for c in prev_state(n-1,q[1:]):  # Recursive Function to determine all the previous state for a given/current state.
            temp.append(q[0:1]+c)
            if q[0] == 1:
                temp.append([0]+c)
            elif q[0] == 2:
                temp.append([1]+c)
    return temp

# To update total number of strings between current and final node, as well as all the child node for the given current node.
def upd(dict,p,q):
    if str(p) in dict.keys():
        if q in dict[str(p)]:
            return dict[str(p)]
        else: 
            val = dict[str(p)][0] + dict[str(q)][0] 
            lis = [val,q] + dict[str(p)][1:]
            return lis
    else:
        return(dict[str(q)][0:1] + [q])

# To get only the total number of strings between current and final node
def clean(dict):
    temp = {}
    for i in dict:
        temp[i] = dict[i][0]
    return temp

# To calculate the conditional probability for between two adjacent states/events in the automaton.
def transPr(q0,q1,n):
    temp = count(n)
    if q1 in str(temp[q0]):
        c0 = temp[q0][0]
        c1 = temp[q1][0]
        return(str(c1) + "/" + str(c0))
    else:
        return(0)

# Definning dictionary to store State as key and the number of child nodes and child state as value. 
def count(n):   # dictionary  state:[number,next...next']
    global fstate,istate,tstate
    tstate = str(final_state(n))
    fstate = str(final_state(n)) +" "+'1' 
    temp = { str(final_state(n)) : [1] }
    back = [final_state(n)]
    for i in range(2*n):# Number of actions possible for a given interval/event 'n'
        new = []
        for state in back:
            more = prev_state(n,state)
            for pstate in more:
                temp.update({ str(pstate): upd(temp,pstate,state) })    
            new.extend(more)
        back = new
    return temp

'''
Representing automaton[Part and Full]
'''

# To construct a subset of States and its associated <action:childState> pair.
def subState(dict,currentState, tempStorage):
    for i in currentState:            
        if str(i) in tempStorage.keys():
            currentState.remove(i)
            continue
        else:
            tempStorage[str(i)] = dict[str(i)]
            currentState.extend(dict[str(i)][1:])
    if len(currentState) != 0:
        subState(dict,currentState, tempStorage)
    return tempStorage

#  To extract inital state of an automaton for the given noOfEvents.
def startState(x):
    temp = x[0]
    return temp

# To extract all possible state of an automaton for the given noOfEvents.
def possibleState(dict):
    temp = [] 
    for i in dict.keys():
        temp += [i]
    return temp

# To modify the extracted possible state of an automaton for the given noOfEvents.
def allState(dict):
    temp = [] 
    for i in dict.keys():
        temp += [i +" " + str(dict[i])]
    return temp

# To construct a dictionary to store actions and destinations possible for all possible state in the automaton 
# for the given noOfEvents.
def transition(dict):
    temp = {}
    for i in dict:
        trans = set()
        if i == tstate:
            continue;
            #temp[i + " " + '1'] = {"1": {str(tstate) +" "+"1"}}
        else: 
            for s in dict[i][1:]:
                trans.add(str(s) + " " + str(dict[str(s)][0]))
            temp[i + " " + str(dict[i][0])] = {".": trans}
    return temp

# To convert input string to list.
def listify(arg):
    return arg if isinstance(arg, list) else [arg]
    
'''
Initializing functions responsible for visualization.
'''

'''
Convert the pdf into jpg format 
''' 
def pdfConvertor():
    pages = convert_from_path('C:/Users/joshu/Digraph.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton.jpg', 'JPEG')
        
'''
Retieve & Display the possible states for the given number of events 
'''        

def retrieveStates(root):   
    clear_canvas(root)
    Label(root, text = "The possible states are:").place(x = 0, y = 25)  #label
    Label(root, text = "Here 0: Unborn 1: Living & 2:Dead").place(x = 0, y = 50) 
    n = noOfEvents.get()
    z = 0
    j = 0
    for i,s in enumerate(possibleState(clean(count(n)))):
        f = Frame(root, height=20, width=100)
        f.pack_propagate(0) # don't shrink
        if (i+1)%10 == 0:
            z = z + 25
            j = 0
        j = j + 1
        f.place(x=100+((j)*100), y= z + 100)       
        label = Label(f, text = s, borderwidth = 1, relief = "solid", bg = 'linen')
        label.pack(fill=BOTH, expand=True)

'''
Retieve & Display the conditional Probability between the given two states: 
'''        

def retrieveProbability(root):   
    clear_canvas(root)
    state1 = childState.get()
    state2 = parentState.get()
    n = noOfEvents.get()
    ans = transPr(state2,state1,n)
    breakValue.set("The Conditional probability between {} and {} is given by:".format(state1,state2))
    Label(root, textvariable = breakValue).place(x = 0, y = 25)  #label 
    Label(root, text = ans, font="Calibri 20").place(x=5, y=50)

'''
Construct & Display the automaton for the given state:
'''

def drawAutomaton1(root):
    tempStorage = {}
    curNode = childState.get()
    s1 = listify(curNode)
    n = noOfEvents.get()
    fState = str(final_state(n))
    vv = subState(count(n),s1, tempStorage)
    if curNode == fState:
        gra = Digraph(format = 'png')
        gra.node('a', fState ,shape='doublecircle')
        newWindow = Toplevel(root)
        newWindow.title("Automaton for" + " "+ childState.get())
        newWindow.geometry("1500x1000+30+30")
        Label(newWindow, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
        gra.render('fState.gv', view=True)
        try:
            image = Image.open("C:/Users/joshu/fState.gv.png")
            image = image.resize((400, 350), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(image)
        except:
            print('Issue in drawing automaton')
    else:    
        nfa1 = VisualNFA(
        states= set(allState(clean(vv))),
        input_symbols={'.'},
        transitions=transition(vv),
        initial_state=startState(allState(clean(vv))),
        final_states={fstate},
        )
        nfa1 = VisualNFA(nfa1)
        nfa1.show_diagram(view = True)
        pdfConvertor()
        newWindow = Toplevel(root)
        newWindow.title("Automaton for" + " "+ childState.get())
        newWindow.geometry("1500x1000+30+30")
        Label(newWindow, text = "The Transition Table:", font="Calibri 15").place(x=5, y=50)
        Label(newWindow, text = nfa1.table, font="Calibri 10").place(x=5, y=125)
        Label(newWindow, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
        try:
            image = Image.open("C:/Users/joshu/automaton.jpg")
            image = image.resize((500, 450), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(image)
        except:
            print('Issue in drawing automaton')
    panel = tk.Label(newWindow, image = img).place(x=5, y=300)
    panel.pack()

def drawAutomaton2(root):
    tempStorage = {}
    s2 = listify(parentState.get())
    n = noOfEvents.get()
    ss = subState(count(n),s2, tempStorage)
    nfa = VisualNFA(
    states= set(allState(clean(ss))),
    input_symbols={'.'},
    transitions=transition(ss),
    initial_state=startState(allState(clean(ss))),
    final_states={fstate},
    )
    nfa = VisualNFA(nfa)
    nfa.show_diagram(view = True)
    pdfConvertor() 
    newWindow1 = Toplevel(root)
    newWindow1.title("Automaton for" + " "+ parentState.get())
    newWindow1.geometry("1500x1000+30+30")
    Label(newWindow1, text = "The Transition Table:", font="Calibri 15").place(x=5, y=50)
    Label(newWindow1, text = nfa.table, font="Calibri 10").place(x=5, y=125)
    Label(newWindow1, text = "The Transition Diagram", font="Calibri 15").place(x=5, y=250)
    try:
        image = Image.open("C:/Users/joshu/automaton.jpg")
        image = image.resize((500, 450), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel = tk.Label(newWindow1, image = img).place(x=5, y=300)
    panel.pack()   

'''
Clear Frame
'''

def clear_canvas(frame):
    for widget in frame.winfo_children():
        widget.destroy()

'''
Initializing tkinter and canvas
'''

root = Tk()
root.geometry("1500x1000+30+30")
root.title("Conditional Probability Calculation for a given number of events in {U,L,D}")
canvas = Canvas(root)
noOfEvents = IntVar()
childState = StringVar()
parentState = StringVar()
breakValue = StringVar()


Label(root, text="ENTER THE TOTAL NUMBER OF EVENTS:", font="Calibri 12").place(x=10, y=0)  #label
Label(root, text="No of Events[Any Integer value greater than 1]").grid(row = 0, padx=(10, 10), pady = (25,0))  #label
Entry(root, textvariable = noOfEvents, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=3, pady = (25,0)) #entry textbox

fr = Frame(root, height=400, width=1350, highlightbackground="blue", highlightthickness=1)
fr.pack_propagate(0) # don't shrink
fr.place(x=75, y=80)
WSignUp = Button(root, text="Get the list of states in the automaton", command= lambda: retrieveStates(fr)).grid(row=3, column=0, padx=(10, 10), sticky=W)
button = Button(root, text="Clear", command= lambda: clear_canvas(fr)).grid(row=4, column=0, padx=(10, 10),  sticky=W)
Label(root, text="ENTER TWO CONSECUTIVE STATES TO CALCULATE THE CONDITIONAL PROBABILITY BETWEEN THEM:", font="Calibri 12").place(x=10, y=500)  #label
fr1 = Frame(root, height=150, width=1350, highlightbackground="blue", highlightthickness=1)
fr1.pack_propagate(0) # don't shrink
fr1.place(x=75, y=630)
Label(root, text="Parent State").place(x = 10, y = 525)   #label
Entry(root, textvariable = parentState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=175, y = 525)#entry textbox
buttonS1 = Button(root, text="Display Automaton", command= lambda: drawAutomaton2(root)).place(x=375, y = 525)
Label(root, text="Child State").place(x=10, y=555)   #label
Entry(root, textvariable = childState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=175, y = 555)
buttonS2 = Button(root, text="Display Automaton", command= lambda: drawAutomaton1(root)).place(x=375, y = 555)
WSignUp1 = Button(root, text="Get the conditional Probability", command = lambda: [retrieveProbability(fr1)]).place(x=10, y = 585)
button1 = Button(root, text="Clear", command= lambda: clear_canvas(fr1)).place(x=10, y = 610)
root.mainloop()